# Sandbox env. for developing middleware classes

A generic dataclass representing a bet on an event

In [2]:
from datetime import datetime
from typing import Set
from pydantic import BaseModel

class Bet(BaseModel):
    """
    Represents a betting event.

    Attributes:
        event_id (int): The unique identifier of the event.
        bookmaker_id (int): The unique identifier of the bookmaker.
        sport (str): The sport of the betting event.
        event_date (datetime): The date and time of the event.
        event_date (datetime): The timezone in which the event occurs.
        participants (set): The set of participants in the event.
        outcome (str): The selected outcome of the bet for the target participant.
        target (int): The participant to which the chosen outcome refers.
        stake (float): The amount wagered on the bet.
        odds (float): The odds of the bet.
        odds_unit (str): The selected odds standard, ie. fractional, american, or decimal.
    """

    event_id: int
    bookmaker_id: int
    sport: str
    event_date: datetime
    event_tz: str
    participants: Set[str]
    outcome: str
    target: str
    stake: float
    odds: float
    odds_unit: str


    @classmethod
    def create(cls, **data):
        """
        Custom class method to create a Bet object.

        Args:
            **data: Keyword arguments representing the attributes of the Bet object.

        Returns:
            Bet: The created Bet object.

        Raises:
            ValueError: If the target participant is not in the participants set.
            ValueError: If the event_tz string is not a valid timezone.
        """
        instance = cls(**data)
        
        # Additional validation
        print(f'post init checks - target: {instance.target}, participants: {instance.participants}')
        if instance.target not in instance.participants:
            raise ValueError("Target participant must be one of the participants")
        
        # You can add more checks for other attributes here
        
        # Check if event_tz is a valid timezone
        from pytz import all_timezones
        if instance.event_tz not in all_timezones:
            raise ValueError("Invalid timezone")
        
        return instance



## Validating class attributes

The below cell is how we expect the class to be created

In [3]:
bet = Bet.create(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Perth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Silly Goose',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal'
)

print(bet)

post init checks - target: Silly Goose, participants: {'Daffy Duck', 'Silly Goose'}
event_id=1 bookmaker_id=1 sport='duck duck goose' event_date=datetime.datetime(2021, 6, 1, 12, 0) event_tz='Australia/Perth' participants={'Daffy Duck', 'Silly Goose'} outcome='win' target='Silly Goose' stake=99.0 odds=1.4 odds_unit='decimal'


Here we test the type checking from `pydantic`

In [4]:
my_instance = Bet.create(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Perth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Silly Goose',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal')
print(my_instance) # Output: MyClass attribute1=10 attribute2='Hello'


# Incorrect usage - raises ValidationError
try:
    my_instance = Bet.create(
        event_id = 1,
        bookmaker_id = 'invalid',
        sport = 'duck duck goose',
        event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
        event_tz = "Australia/Perth",
        participants = {'Silly Goose', 'Daffy Duck'},
        outcome = 'win',
        target = 'Silly Goose',
        stake = 99.0,
        odds = 1.4,
        odds_unit = 'decimal'
        )
    print(my_instance)
except Exception as e:
    print(str(e))

post init checks - target: Silly Goose, participants: {'Daffy Duck', 'Silly Goose'}
event_id=1 bookmaker_id=1 sport='duck duck goose' event_date=datetime.datetime(2021, 6, 1, 12, 0) event_tz='Australia/Perth' participants={'Daffy Duck', 'Silly Goose'} outcome='win' target='Silly Goose' stake=99.0 odds=1.4 odds_unit='decimal'
1 validation error for Bet
bookmaker_id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='invalid', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing


Here we test out `__post_init__` method validations

In [5]:
try:
    my_instance = Bet.create(
        event_id = 1,
        bookmaker_id = 1,
        sport = 'duck duck goose',
        event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
        event_tz = "Australia/Perth",
        participants = {'Silly Goose', 'Daffy Duck'},
        outcome = 'win',
        target = 'Stupid Bird',
        stake = 99.0,
        odds = 1.4,
        odds_unit = 'decimal')
    print(my_instance)
except ValueError as e:
    print(str(e))

post init checks - target: Stupid Bird, participants: {'Daffy Duck', 'Silly Goose'}
Target participant must be one of the participants


In [7]:
# Incorrect usage - raises ValidationError
try:
    my_instance = Bet.create(
        event_id = 1,
        bookmaker_id = 1,
        sport = 'duck duck goose',
        event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
        event_tz = "Australia/Purth",
        participants = {'Silly Goose', 'Daffy Duck'},
        outcome = 'win',
        target = 'Silly Goose',
        stake = 99.0,
        odds = 1.4,
        odds_unit = 'decimal'
        )
    print(my_instance)
except Exception as e:
    print(str(e))

post init checks - target: Silly Goose, participants: {'Daffy Duck', 'Silly Goose'}
Invalid timezone


Let's restrict direct instantiation of the class so that we **must** use the create method to create an instance of the class. We'll create a singleton class that inherits from BaseModel, and then a custom base model that inherits from both of these classes, our Bet class will then inherit from this CustomBaseModel

In [31]:
from datetime import datetime
from typing import Set
from pydantic import BaseModel


class NoDirectInstantiationMeta(type(BaseModel)):
    """
    A metaclass for preventing direct instantiation of classes.
    """
    def __call__(cls, *args, **kwargs):
        raise ValueError("This class cannot be instantiated directly. Use the create() method instead.")
    

class CustomBaseModel(BaseModel, metaclass=NoDirectInstantiationMeta):
    pass


class Bet(CustomBaseModel):
    """
    Represents a betting event.

    Attributes:
        event_id (int): The unique identifier of the event.
        bookmaker_id (int): The unique identifier of the bookmaker.
        sport (str): The sport of the betting event.
        event_date (datetime): The date and time of the event.
        event_date (datetime): The timezone in which the event occurs.
        participants (set): The set of participants in the event.
        outcome (str): The selected outcome of the bet for the target participant.
        target (int): The participant to which the chosen outcome refers.
        stake (float): The amount wagered on the bet.
        odds (float): The odds of the bet.
        odds_unit (str): The selected odds standard, i.e., fractional, american, or decimal.
    """

    event_id: int
    bookmaker_id: int
    sport: str
    event_date: datetime
    event_tz: str
    participants: Set[str]
    outcome: str
    target: str
    stake: float
    odds: float
    odds_unit: str

    def __str__(self):
        return (
            f"""
            Bet(
                event_id={self.event_id},
                f"bookmaker_id={self.bookmaker_id},
                f"sport='{self.sport}',
                f"event_date={self.event_date.strftime('%Y-%m-%d %H:%M:%S')},
                f"event_tz='{self.event_tz}',
                f"participants={', '.join(self.participants)},
                f"outcome='{self.outcome}',
                f"target='{self.target}',
                f"stake={self.stake:.2f},
                f"odds={self.odds:.2f},
                f"odds_unit='{self.odds_unit}'
            )"""
        )

    @classmethod
    def create(cls, **data):
        """
        Custom class method to create a Bet object.

        Args:
            **data: Keyword arguments representing the attributes of the Bet object.

        Returns:
            Bet: The created Bet object.

        Raises:
            ValueError: If the target participant is not in the participants set.
            ValueError: If the event_tz string is not a valid timezone.
        """
        instance = super().__new__(cls)
        instance.__init__(**data)

        print(f'post init checks - target: {instance.target}, participants: {instance.participants}')
        if instance.target not in instance.participants:
            raise ValueError("Target participant must be one of the participants")

        # You can add more checks for other attributes here

        # Check if event_tz is a valid timezone
        from pytz import all_timezones
        if instance.event_tz not in all_timezones:
            raise ValueError("Invalid timezone")

        return instance
    
    def place_with_bookmaker(self, bookmaker_api):
        """
        Place this bet using a specific bookmaker's API.

        Args:
            bookmaker_api: An instance of a subclass of BookmakerAPI.
        """
        # Ensure that bookmaker_api is authenticated before placing the bet
        if not bookmaker_api.is_authenticated:
            bookmaker_api.authenticate()
        
        # Use bookmaker API to place the bet
        # Retrieve this instance's attributes as a dictionary
        bet_details = vars(self)
        
        # # Remove attributes that the API doesn't accept
        # unwanted_keys = ['event_date', 'event_tz', 'participants', 'odds_unit']
        # for key in unwanted_keys:
        #     bet_details.pop(key, None)

        # # You may need to transform some attributes if the API expects different names or formats
        # bet_details['bet_type'] = bet_details.pop('outcome')  # Example of renaming an attribute

        # Use the ** operator to unpack the dictionary into keyword arguments
        bookmaker_api.place_bet(**bet_details)


new_instance = Bet.create(
    event_id=1,
    bookmaker_id=1,
    sport='duck duck goose',
    event_date=datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz="Australia/Perth",
    participants={'Silly Goose', 'Daffy Duck'},
    outcome='win',
    target='Silly Goose',
    stake=99.0,
    odds=1.4,
    odds_unit='decimal'
)
print(my_instance)

# Attempting to instantiate directly will raise an error
try:
    my_instance_direct = Bet(
        event_id=2,
        bookmaker_id=2,
        sport='duck duck goose',
        event_date=datetime.now(),
        event_tz="Australia/Perth",
        participants={'Silly Goose', 'Daffy Duck'},
        outcome='win',
        target='Silly Goose',
        stake=99.0,
        odds=1.4,
        odds_unit='decimal'
    )
except ValueError as e:
    print(str(e))


post init checks - target: Silly Goose, participants: {'Daffy Duck', 'Silly Goose'}
event_id=1 bookmaker_id=1 sport='duck duck goose' event_date=datetime.datetime(2021, 6, 1, 12, 0) event_tz='Australia/Perth' participants={'Daffy Duck', 'Silly Goose'} outcome='win' target='Silly Goose' stake=99.0 odds=1.4 odds_unit='decimal'
This class cannot be instantiated directly. Use the create() method instead.


Now if I try to create another instance using the create method

In [12]:
new_instance = Bet.create(
    event_id=1,
    bookmaker_id=1,
    sport='duck duck goose',
    event_date=datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz="Australia/Perth",
    participants={'Silly Goose', 'Daffy Duck'},
    outcome='win',
    target='Silly Goose',
    stake=99.0,
    odds=1.4,
    odds_unit='decimal'
)
print(my_instance)

post init checks - target: Silly Goose, participants: {'Daffy Duck', 'Silly Goose'}
event_id=1 bookmaker_id=1 sport='duck duck goose' event_date=datetime.datetime(2021, 6, 1, 12, 0) event_tz='Australia/Perth' participants={'Daffy Duck', 'Silly Goose'} outcome='win' target='Silly Goose' stake=99.0 odds=1.4 odds_unit='decimal'


## Adaptor class
Abstract base class of adaptors for bookmakers API's. The main relevant methods are here

In [15]:
from abc import ABC, abstractmethod

class BookmakerAPI(ABC):
    """
    Abstract base class to represent a bookmaker's API interface.
    """
    is_authenticated = False
    
    @abstractmethod
    def authenticate(self):
        """
        Authenticate with the bookmaker's API service.

        Implementations should establish a session or retrieve an auth token.
        """
        pass

    @abstractmethod
    def get_available_events(self):
        """
        Retrieve available betting events from the bookmaker.

        Implementations should return data in a consistent format.
        """
        pass

    @abstractmethod
    def place_bet(self, event_id, stake, odds, bet_type="single", **kwargs):
        """
        Place a bet on an event with the bookmaker.

        Args:
            event_id: The identifier for the event to bet on.
            stake: The amount of money to wager.
            odds: The agreed upon odds at the time of placing the bet.
            bet_type: The type of bet (e.g., single, parlay, etc.).
            **kwargs: Additional arguments specific to the bet type or bookmaker.

        Implementations should perform the bet placement and handle errors.
        """
        pass

    @abstractmethod
    def withdraw_funds(self, amount):
        """
        Withdraw funds from the user's account.

        Args:
            amount: The amount of money to withdraw.

        Implementations should carry out the withdrawal process.
        """
        pass

Say we want to implement this class for 'SportsBet AU'

In [40]:
class SportsBetAU(BookmakerAPI):
    def __init__(self):
        self.is_authenticated = False
        # Initialize other necessary data for AcmeBookmaker

    def authenticate(self):
        # Implement authentication logic for Acme Bookmaker's API
        # If successful, set self.is_authenticated to True
        self.is_authenticated = True
        print("Authenticated with SportsBetAU.")

    def get_available_events(self):
        # Implement the logic to retrieve available events from Acme Bookmaker's API
        pass
    
    def place_bet(self, 
                  event_id,
                  bookmaker_id,
                  sport,
                  event_date,
                  event_tz,
                  participants,
                  outcome,
                  target,
                  stake,
                  odds,
                  odds_unit
                  ):

        if not self.is_authenticated:
            print("Access denied: User is not authenticated.")
            return  # Exit the method if not authenticated

        # Implement the logic to place a bet through Acme Bookmaker's API
        print(f"Placing bet on event {event_id} with stake {stake} at odds {odds}.")
        
        # Summary print statement
        participants_str = ', '.join(participants)  # Assuming participants is a list of strings

        print(f"🎲 Bet summary:")
        print(f"  Event ID: {event_id}")
        print(f"  Bookmaker ID: {bookmaker_id}")
        print(f"  Sport: {sport}")
        print(f"  Event Date & Timezone: {event_date} ({event_tz})")
        print(f"  Participants: {participants_str}")
        print(f"  Outcome Wagered: {outcome}")
        print(f"  Target condition (if applicable): {target}")
        print(f"  Stake: {stake}")
        print(f"  Odds: {odds} {odds_unit}")

    def withdraw_funds(self, amount):
        
        if not self.is_authenticated:
            print("Access denied: User is not authenticated.")
            return  # Exit the method if not authenticated
        # Implement the logic to withdraw funds from Acme Bookmaker's API
        pass

# Usage of the concrete class
sports_bet_AU = SportsBetAU()
# acme_bookmaker.authenticate()  # Would authenticate the user
# events = acme_bookmaker.get_available_events()  # Would get available events


In [41]:
new_bet = Bet.create(
    event_id=1,
    bookmaker_id=1,
    sport='duck duck goose',
    event_date=datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz="Australia/Perth",
    participants={'Silly Goose', 'Daffy Duck'},
    outcome='win',
    target='Silly Goose',
    stake=99.0,
    odds=1.4,
    odds_unit='decimal'
)
print(new_bet)

post init checks - target: Silly Goose, participants: {'Daffy Duck', 'Silly Goose'}

            Bet(
                event_id=1,
                f"bookmaker_id=1,
                f"sport='duck duck goose',
                f"event_date=2021-06-01 12:00:00,
                f"event_tz='Australia/Perth',
                f"participants=Daffy Duck, Silly Goose,
                f"outcome='win',
                f"target='Silly Goose',
                f"stake=99.00,
                f"odds=1.40,
                f"odds_unit='decimal'
            )


In [42]:
sports_bet_AU = SportsBetAU()  # Create an instance of a specific bookmaker

# Place the bet with the chosen bookmaker
new_bet.place_with_bookmaker(sports_bet_AU)


Authenticated with SportsBetAU.
Placing bet on event 1 with stake 99.0 at odds 1.4.
🎲 Bet summary:
  Event ID: 1
  Bookmaker ID: 1
  Sport: duck duck goose
  Event Date & Timezone: 2021-06-01 12:00:00 (Australia/Perth)
  Participants: Daffy Duck, Silly Goose
  Outcome Wagered: win
  Target condition (if applicable): Silly Goose
  Stake: 99.0
  Odds: 1.4 decimal
